# Data Wrangling WeRateDogs

## Gathering

* **Twitter archive file** Manually download by clicking the link: 
* **Tweet Image prediction** Programtically download the file 
* **Tweet JSON file** Get from twitter using API. 

In [1]:
#Imports 
import pandas as pd 
import numpy as np 
import matplotlib.pyplot as plt
%matplotlib inline 
import requests 
import tweepy
import json
import time

### 1 Twitter archive 

In [2]:
twitter_archive = pd.read_csv('twitter-archive-enhanced.csv')

### 2 Tweet Image prediction

In [3]:
url = 'https://d17h27t6h515a5.cloudfront.net/topher/2017/August/599fd2ad_image-predictions/image-predictions.tsv'
page = requests.get(url)
with open('image_predictions.tsv', mode = 'wb') as f: 
    f.write(page.content)
image_prediction = pd.read_csv('image_predictions.tsv', sep = '\t')

### 3 Tweet JSON file

In [4]:
consumer_key = 'IlUpuNICtvcgHR66TrBLN43k7'
consumer_secret = 'WZXOLRl7sBUpfIBeoukENQ5Z0ZcfPUOB6ouyxSrTYwNcRTXhSs'
access_token = '1307275021522530306-jmWn1Fm4mu7i431hpL4gnMsd3dTCfY'
access_secret = 'NtlNyDFUgIPFcTxlFDhEPzSeVRAv1vtQLm61dDIBPzFZ5'

In [5]:
auth = tweepy.OAuthHandler(consumer_key, consumer_secret)
auth.set_access_token(access_token, access_secret)
api = tweepy.API(auth, wait_on_rate_limit = True, wait_on_rate_limit_notify = True)

In [6]:
#Create a list of the tweet ids from the image_prediction DataFrame. 
tweet_ids = list(image_prediction['tweet_id'])

In [7]:
#quick test to check if we can pull tweets correctly. 
tweet = api.get_status(tweet_ids[0], tweet_mode = 'extended')
print(tweet.full_text)

Here we have a Japanese Irish Setter. Lost eye in Vietnam (?). Big fan of relaxing on stair. 8/10 would pet https://t.co/BLDqew2Ijj


In [8]:
error_log = {}
start = time.time()
with open('tweet_json.txt', 'w') as outfile: 
    for index, tweet in image_prediction.iterrows():
        try:
            tweet_json = api.get_status(tweet.tweet_id, tweet_mode = 'extended')._json
            json.dump(tweet_json, outfile)
            outfile.write('\n')
        except tweepy.TweepError as e:
                error_log[tweet.tweet_id] =e
                pass
end = time.time()
print('process complete, time taken: {}'.format(end-start))

with open('error_log.txt', 'w') as filehandle:
    for error in error_log:
        filehandle.write('%s\n' % error)


Rate limit reached. Sleeping for: 698
Rate limit reached. Sleeping for: 699


process complete, time taken: 1871.3138456344604


In [18]:
tweet_jsons = []
jsons_file = open('tweet_json.txt', 'r')
for line in jsons_file:
    try: 
        tweet = json.load(line)
        tweet_jsons.append(tweet)
    except:
        pass

jsons_file.close()

In [19]:
print(tweet_jsons)

[]


In [ ]:
tweet_df.sample(10)

In [ ]:
image_prediction.sample(10)

In [ ]:
twitter_archive.sample(10)

### Programmatic assessment

In [ ]:
tweet_df.info()

In [ ]:
tweet_df.describe()

In [ ]:
image_prediction.info()

In [ ]:
image_prediction.describe()

In [ ]:
twitter_archive.info()

In [ ]:
twitter_archive.describe()

### Main issues

#### Quality issues 
* twitter_archive: source column is in HTML format.
* image_prediction: some dog names in p1, p2 and p3 start with capitals and others don't. 
* 
* 
* 
* 
* 
* 

#### Tidiness issues

* twitter_archive : 4 columns for the different dog stages, when only one is necessary.
* 

In [ ]:
#first, create copies of the DataFrames
twitter_archive_clean = twitter_archive.copy()
image_prediction_clean = image_prediction.copy()
tweet_df_clean = tweet_df.copy()

*twitter_archive: source column format is in HTML*
> Do some string parsing on this column to retain the URL

In [ ]:
twitter_archive_clean = twitter_archive.copy()

In [ ]:
twitter_archive_clean['source'] = twitter_archive_clean['source'].str.split('"')[1]

In [ ]:
twitter_archive_clean['source'][1]

*twitter_archive : 4 columns for the different dog stages, when only one is necessary.*
> Create a new column dog_stage which melts these 4 stages. Then drop the columns.

In [ ]:
twitter_archive_clean.columns

In [ ]:
melt_cols = ['doggo', 'floofer', 'pupper', 'puppo']
kept_cols = [c for c in list(twitter_archive_clean.columns) if c not in melt_cols]

In [ ]:
twitter_archive_clean = pd.melt(twitter_archive_clean, id_vars = kept_cols, value_vars= melt_cols, var_name = 'stages', value_name ='dog_stage')
twitter_archive_clean.drop('stages', axis= 1, inplace= True)

##### Check

In [ ]:
twitter_archive_clean.head()